In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# df = pd.read_parquet('filtered_data.parquet')
df = pd.read_parquet('full_data.parquet')

In [3]:
start = df['ts'].min()
end = df['ts'].max()

new_start = int((end + start)/2)

In [4]:
print('START:    ', datetime.utcfromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S'))
print('END:      ', datetime.utcfromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S'))
print('NEW START:', datetime.utcfromtimestamp(new_start).strftime('%Y-%m-%d %H:%M:%S'))

START:     2022-03-01 00:00:00
END:       2022-08-31 23:59:58
NEW START: 2022-05-31 23:59:59


In [5]:
df = df[df['ts'] > new_start]
df

,user_id,track_id,session_id,ts
0,2221415,18127,559,1658376527
1,1296562,36071,374,1654085603
2,1698556,4031,486,1656494630
5,2768742,29092,462,1661071150
6,2769174,9082,640,1660206793
...,...,...,...,...
704728513,2954562,14061,553,1655020035
704728514,1301303,14061,310,1659614629
704728516,1141809,14061,392,1658668024
704728517,2436350,14061,400,1654285059


In [6]:
df['user_id'].value_counts()

user_id
154350     3640
587363     3500
95935      3460
1510916    3370
2338704    3350
           ... 
2021152       1
2760757       1
149825        1
930915        1
1844664       1
Name: count, Length: 2906303, dtype: int64

In [7]:
# top_user = 5000
# top_item = 1000

In [8]:
# user_counts = df['user_id'].value_counts()
# user_counts = user_counts.iloc[:top_user].index
# item_counts = df['track_id'].value_counts()
# item_counts = item_counts.iloc[:top_item].index

In [9]:
# df = df[(df['user_id'].isin(user_counts.to_numpy())) & (df['track_id'].isin(item_counts.to_numpy()))]

In [10]:
user_core = 1000
item_core = 1000

filter_more = True
filter_id = 0


while filter_more:
    filter_id = filter_id + 1
    
    print(f'Filtering round number {filter_id}')
    filter_more = False

    print('\tFiltering users')
    
    ucounts = df['user_id'].value_counts()

    ucounts_filt = ucounts[ucounts >= user_core]

    if len(ucounts) != len(ucounts_filt):
        print('\t\tWill filter once more because of users.')
        filter_more = True

    print('\tFiltering items')

    icounts = df['track_id'].value_counts()

    icounts_filt = icounts[icounts >= item_core]

    if len(icounts) != len(icounts_filt):
        print('\t\tWill filter once more because of items.')
        filter_more = True

    print('\tFiltering the whole dataset')

    df = df[(df['user_id'].isin(ucounts_filt.index.to_numpy())) & (df['track_id'].isin(icounts_filt.index.to_numpy()))]

    print(f'\tSaving filtered dataset with {len(df)} interactions')
    df[['user_id', 'track_id', 'ts']].to_parquet('done_filtered_data.parquet', index=False)

Filtering round number 1
	Filtering users
		Will filter once more because of users.
	Filtering items
		Will filter once more because of items.
	Filtering the whole dataset
	Saving filtered dataset with 28152515 interactions
Filtering round number 2
	Filtering users
		Will filter once more because of users.
	Filtering items
		Will filter once more because of items.
	Filtering the whole dataset
	Saving filtered dataset with 25086549 interactions
Filtering round number 3
	Filtering users
		Will filter once more because of users.
	Filtering items
		Will filter once more because of items.
	Filtering the whole dataset
	Saving filtered dataset with 20432396 interactions
Filtering round number 4
	Filtering users
		Will filter once more because of users.
	Filtering items
		Will filter once more because of items.
	Filtering the whole dataset
	Saving filtered dataset with 19729701 interactions
Filtering round number 5
	Filtering users
		Will filter once more because of users.
	Filtering items
		W

In [11]:
df['user_id'].value_counts()

user_id
95935      3438
1510916    3222
154350     3202
3199213    3135
1355597    3115
           ... 
34389      1000
2368832    1000
1269653    1000
2742668    1000
2953266    1000
Name: count, Length: 13209, dtype: int64

In [12]:
df['track_id'].value_counts()

track_id
14061    143951
42052    133891
26996     99379
27887     96098
46999     86907
          ...  
26781      1004
36116      1001
37912      1001
21183      1001
15157      1000
Name: count, Length: 3027, dtype: int64

In [13]:
df.sort_values(by=['user_id', 'ts'], inplace=True)

In [14]:
df['user_id'].unique()

array([      5,      83,     105, ..., 3420720, 3422309, 3425484],
      shape=(13209,))

In [15]:
user_map = {x : i+1 for i,x in enumerate(df['user_id'].unique())}
item_map = {x : i+1 for i,x in enumerate(df['track_id'].unique())}

In [18]:
df['user_id'] = df['user_id'].map(user_map.get)
df['track_id'] = df['track_id'].map(item_map.get)

In [21]:
df[['user_id', 'track_id', 'ts']].to_parquet('sorted_data.parquet', index=False)